# **Predicting the XRP Price using Geometric Brownian Motion for 30 days**

In [49]:
import pandas as pd
import numpy as np

In [50]:
import plotly.graph_objects as go
from plotly.offline import plot

In [51]:
from scipy import stats

In [52]:
crypto_name = "binancecoin"

In [53]:
# Load stock prices from CSV
file_path = "C:\\Heri\\GitHub\\Crypto\\02-Coingecko\\data-crypto.csv"
data = pd.read_csv(file_path)

In [54]:
# Assuming the CSV has a column named 'Price'
prices = data[crypto_name]  # Replace with your column name

In [55]:
# Parameters for simulation
start_price = prices.iloc[-1]  # Start from the last known price
time_horizon = 30  # Days
steps = 3000  # Simulation steps

In [56]:
# Calculate daily returns
daily_returns = prices.pct_change().dropna()

In [57]:
# Estimate parameters for Brownian motion
mu = daily_returns.mean()  # Mean return
sigma = daily_returns.std()  # Volatility

## **Geometric Brownian Motion**

In [58]:
# Simulate future prices using geometric Brownian motion
def simulate_gbm(start_price, mu, sigma, time_horizon, steps):
    dt = time_horizon / steps
    price_path = [start_price]
    for _ in range(steps):
        price_path.append(price_path[-1] * np.exp((mu - 0.5 * sigma ** 2) * dt +
                                                  sigma * np.sqrt(dt) * np.random.normal()))
    return price_path

In [59]:
# Run simulation
simulated_prices = simulate_gbm(start_price, mu, sigma, time_horizon, steps)

In [60]:
# Create the plot
fig = go.Figure()

# Add the simulated prices line
fig.add_trace(go.Scatter(
    x=list(range(len(simulated_prices))),
    y=simulated_prices,
    mode='lines',
    name='Simulated Prices',
    line=dict(color='#206583', width=2)
))

# Update layout for aesthetics
fig.update_layout(
    title={
        'text': f'GBM for {time_horizon} Days for {crypto_name}',
        'font': {
            'size': 30
        }
    },
    xaxis_title='Time Step',
    yaxis_title='Price',
    font=dict(size=14),
    plot_bgcolor='white',
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey')
)

In [61]:
# Render the plot in HTML
plot(fig, auto_open=False, filename='Average Crypto Market Price Prediction.html')

'Average Crypto Market Price Prediction.html'

## **Multiple Geometric Brownian Motion**

In [62]:
# Run simulation 10 times
simulations = []

simulations.append(simulated_prices)

for simulation in range(14):
    simulated_prices = simulate_gbm(start_price, mu, sigma, time_horizon, steps)
    simulations.append(simulated_prices)

In [63]:
# Create the plot
fig = go.Figure()

# Define a list of colors
colors = ['#0D3239', '#1A5465', '#206583', '#3487AB', '#3487AB', '#0D3239', '#113C46', '#184F5F', '#1D5B72', '#1E607A', '#266F8E', '#3182A5', '#3487AB', '#3487AB', '#3487AB']

# Add each simulated prices line with different colors
for i, prices in enumerate(simulations):
    fig.add_trace(go.Scatter(
        x=list(range(len(prices))),
        y=prices,
        mode='lines',
        name=f'Simulation {i+1}',
        line=dict(color=colors[i % len(colors)], width=1)
    ))

# Update layout for aesthetics
fig.update_layout(
    title={
        'text': f'GBM for {time_horizon} Days for {crypto_name}',
        'font': {
            'size': 30
        }
    },
    xaxis_title='Time Step',
    yaxis_title='Price',
    font=dict(size=14),
    plot_bgcolor='white',
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey')
)

In [64]:
# Render the plot in HTML
plot(fig, auto_open=False, filename='Average Crypto Market Price Multiple Prediction using GBM for 1 Year.html')

'Average Crypto Market Price Multiple Prediction using GBM for 1 Year.html'

## **Probability of the Last Price Being Greater than the Initial Price**

In [65]:
for simulation in range(2000):
    simulated_prices = simulate_gbm(start_price, mu, sigma, time_horizon, steps)
    simulations.append(simulated_prices)

In [66]:
# Get last values
final_values = [sim[-1] for sim in simulations]

# Get initial value
initial_value = simulations[0][0]

In [67]:
count_greater = sum(1 for value in final_values if value > initial_value)
total_values = len(final_values)
probability = count_greater / total_values if total_values > 0 else 0

print(f'Initial Value: {initial_value}')
print(f'Count Greater: {count_greater}')
print(f'Total Values: {total_values}')
print(f'Probability: {probability}')


Initial Value: 669.0228793928828
Count Greater: 1328
Total Values: 2015
Probability: 0.6590570719602977


## **Distribution of the Last Price using Geometric Brownian Motion**

In [68]:
# Convert list to pandas Series
data_series = pd.Series(final_values)
print(data_series.describe())

count    2015.000000
mean      731.270522
std       123.834986
min       416.366402
25%       643.441411
50%       722.609319
75%       809.812066
max      1270.453330
dtype: float64


In [69]:
# Fit a gamma distribution to the data
shape, loc, scale = stats.gamma.fit(data_series)

# Print the estimated parameters
print(f"Shape: {shape}, Location: {loc}, Scale: {scale}")

Shape: 12.939678496290691, Location: 285.9773739820398, Scale: 34.41299935691994


In [70]:
# Calculate statistics
mean = np.mean(final_values)
std = np.std(final_values)
skew = stats.skew(final_values)
kurtosis = stats.kurtosis(final_values)
median = np.median(final_values)

In [71]:
# Create histogram
hist_data = go.Histogram(
    x=final_values,
    nbinsx=200,
    histnorm='probability density',
    marker=dict(color='skyblue', opacity=0.7),
    name='Histogram'
)

# Add normal distribution curve
x = np.linspace(0, max(final_values), 100)

# Calculate the gamma distribution curve
gamma_curve = go.Scatter(
    x=x,
    y=stats.gamma.pdf(x, shape, loc, scale),
    mode='lines',
    line=dict(color='blue', width=2),
    name='Gamma Distribution'
)

# Add vertical line for initial value
initial_value_line = go.Scatter(
    x=[initial_value, initial_value],
    y=[0, max(stats.norm.pdf(x, mean, std))],
    mode='lines',
    line=dict(color='green', width=2, dash='dash'),
    name='Initial Value'
)

# Create figure
fig = go.Figure(data=[hist_data, gamma_curve, initial_value_line])

# Add statistics annotation
fig.add_annotation(
    xref='paper', yref='paper',
    x=0.05, y=0.95,
    text=f'Mean: {mean:.2f}<br>Std: {std:.2f}<br>Skew: {skew:.2f}<br>Kurtosis: {kurtosis:.2f}<br>Median: {median:.2f}',
    showarrow=False,
    align='left'
)

# Update layout
fig.update_layout(
    title={
        'text': f'Distribution of Final Values for {crypto_name} in {time_horizon} Days',
        'font': {
            'size': 30
        }
    },
    xaxis_title='Final Price',
    yaxis_title='Density',
    legend=dict(x=0.7, y=0.95),
    plot_bgcolor='white',
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey')
)

In [72]:
# Render the plot in HTML
plot(fig, auto_open=False, filename='Average Crypto Market Distribution using GBM.html')

'Average Crypto Market Distribution using GBM.html'